In [1]:
#filter_breakdown: 


In [17]:
# run this cell
import pynwb
import os
import numpy as np


#DataJoint and DataJoint schema
import datajoint as dj

## We also import a bunch of tables so that we can call them easily
#from nwb_datajoint.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
#                                  DataAcquisitionDevice, CameraDevice, Probe,
#                                  DIOEvents,
#                                  ElectrodeGroup, Electrode, Raw, SampleCount,
#                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
#                                  SortGroup, SpikeSorting, SpikeSorter, SpikeSorterParameters, SpikeSortingWaveformParameters, SpikeSortingParameters, SpikeSortingMetrics, CuratedSpikeSorting,\
#                                  FirFilter,
#                                  IntervalList, SortInterval,
#                                  Lab, LabMember, LabTeam, Institution,
#                                  BrainRegion,
#                                  SensorData,
#                                  Session, ExperimenterList,
#                                  Subject,
#                                  Task, TaskEpoch,
#                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery,
#                                  interval_list_contains,
#                                  interval_list_contains_ind,
#                                  interval_list_excludes,
#                                  interval_list_excludes_ind,
#                                  interval_list_intersect,
#                                  get_electrode_indices)

from nwb_datajoint.common import (RawPosition, HeadDir, Speed, LinPos, StateScriptFile, VideoFile,
                                  DataAcquisitionDevice, CameraDevice, Probe,
                                  DIOEvents,
                                  ElectrodeGroup, Electrode, Raw, SampleCount,
                                  LFPSelection, LFP, LFPBandSelection, LFPBand,
                                  SortGroup, SpikeSortingFilterParameters, SpikeSortingArtifactDetectionParameters,
                                  SpikeSortingRecordingSelection, SpikeSortingRecording, 
                                  SpikeSortingWorkspace, 
                                  SpikeSorter, SpikeSorterParameters, SortingID,
                                  SpikeSortingSelection, SpikeSorting, 
                                  SpikeSortingMetricParameters,
                                  ModifySortingParameters, ModifySortingSelection, ModifySorting, 
                                  AutomaticCurationParameters, AutomaticCurationSelection,
                                  AutomaticCuration,
                                  CuratedSpikeSortingSelection, CuratedSpikeSorting,
                                  UnitInclusionParameters,
                                  FirFilter,
                                  IntervalList, SortInterval,
                                  Lab, LabMember, LabTeam, Institution,
                                  BrainRegion,
                                  SensorData,
                                  Session, ExperimenterList,
                                  Subject,
                                  Task, TaskEpoch,
                                  Nwbfile, AnalysisNwbfile, NwbfileKachery, AnalysisNwbfileKachery,
                                 get_electrode_indices)


import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

In [5]:
from nwb_datajoint.common import get_electrode_indices

#### Next we select the NWB file, which corresponds to the dataset we want to extract LFP from

In [ ]:
# nwb_file_names = Nwbfile().fetch('nwb_file_name')
# # take the first one for this demonstration
# nwb_file_name = nwb_file_names[0]
# print(nwb_file_name)


In [3]:
FirFilter()

filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,filter_type,filter_low_stop lowest frequency for stop band for low frequency side of filter,filter_low_pass lowest frequency for pass band of low frequency side of filter,filter_high_pass highest frequency for pass band for high frequency side of filter,filter_high_stop highest frequency for stop band of high frequency side of filter,filter_comments comments about the filter,filter_band_edges numpy array containing the filter bands (redundant with individual parameters),filter_coeff numpy array containing the filter coefficients
LFP 0-400 Hz,20000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
LFP 0-400 Hz,30000,lowpass,0.0,0.0,400.0,425.0,standard LFP filter for 20 KHz data,=BLOB=,=BLOB=
Ripple 150-250 Hz,1000,lowpass,140.0,150.0,250.0,260.0,ripple band filter for 1 kHz data,=BLOB=,=BLOB=
Theta 5-11 Hz,1000,lowpass,4.0,5.0,11.0,12.0,theta filter for 1 KHz data,=BLOB=,=BLOB=


#### Create the standard LFP Filters. This only needs to be done once.

In [4]:
FirFilter().create_standard_filters()

In [7]:
# run this cell
#nwb_file_name = 'CH6520211201_.nwb'
nwb_file_name = 'beans20190718_emonroe_.nwb'
# print(((Electrode() & {'nwb_file_name' : nwb_file_name}).fetch('electrode_id'))[range(0, , 4)])
# print(((Electrode() & {'nwb_file_name' : nwb_file_name}).fetch('electrode_group_name'))[range(0, 256, 4)])

#[6,8,2,4,7,9,12,20,21,24,25,30,33,35,36,37,38,40,41,44,45,47,
#                        51,52,53,54,55,56,57,58,60,61,]
# intermed. canula LFP electrodes: [6,8,2,4,7,9,12,20,21,24,25,]
# dorsal canula LFP electrodes: []

In [19]:
lfp_file_name = AnalysisNwbfile().create('beans20190718_emonroe_.nwb')
lfp_file_abspath = AnalysisNwbfile().get_abs_path(lfp_file_name)

Writing new NWB file beans20190718_emonroe_IWIV1WRU4C.nwb


In [20]:
rawdata = (Raw().nwb_object({'nwb_file_name' : nwb_file_name}))
eseries=rawdata

In [ ]:
sampling_rate, interval_list_name = (Raw() & {'nwb_file_name' : nwb_file_name}).fetch1('sampling_rate', 'interval_list_name')
filter = (FirFilter() & {'filter_name': 'LFP 0-400 Hz'} &
                  {'filter_sampling_rate': sampling_rate}).fetch(as_dict=True)
filter
filter_coeff = filter[0]['filter_coeff']
filter_coeff



In [24]:
valid_times = (IntervalList() & {'nwb_file_name': 'nwb_file_name',
                                         'interval_list_name': 'lfp valid times'}).fetch1('valid_times')
        # keep only the intervals > 1 second long
min_interval_length = 1.0
valid = []
for count, interval in enumerate(valid_times):
    if interval[1] - interval[0] > min_interval_length:
        valid.append(count)
    valid_times = valid_times[valid] 

DataJointError: fetch1 should only return one tuple. 0 tuples found

In [ ]:
analysis_file_abs_path, eseries, filter_coeff, valid_times, electrode_ids, decimation

In [ ]:
analysis_file_abs_path = 

In [10]:
rawdata = (Raw().nwb_object({'nwb_file_name' : nwb_file_name}))
eseries=rawdata

data_on_disk = rawdata.data



In [11]:
timestamps_on_disk = eseries.timestamps



In [13]:
n_dim = len(data_on_disk.shape)
n_samples = len(timestamps_on_disk)




In [ ]:
time_axis = 0 if data_on_disk.shape[0] == n_samples else 1



In [15]:
electrode_axis = 1 - time_axis
n_electrodes = data_on_disk.shape[electrode_axis]
input_dim_restrictions = [None] * n_dim

In [18]:
input_dim_restrictions[electrode_axis] = np.s_[get_electrode_indices(eseries, electrode_ids)]



NameError: name 'electrode_ids' is not defined

In [2]:
# def filter_data_nwb(self, analysis_file_abs_path, eseries, filter_coeff, valid_times, electrode_ids,
#                         decimation):
#     """
#         :param analysis_nwb_file_name: str   full path to previously created analysis nwb file where filtered data
#         should be stored. This also has the name of the original NWB file where the data will be taken from
#         :param eseries: electrical series with data to be filtered
#         :param filter_coeff: numpy array with filter coefficients for FIR filter
#         :param valid_times: 2D numpy array with start and stop times of intervals to be filtered
#         :param electrode_ids: list of electrode_ids to filter
#         :param decimation: int decimation factor
#         :return: The NWB object id of the filtered data (str), list containing first and last timestamp

#         This function takes data and timestamps from an NWB electrical series and filters them using the ghostipy
#         package, saving the result as a new electricalseries in the nwb_file_name, which should have previously been
#         created and linked to the original NWB file using common_session.AnalysisNwbfile.create()
#         """

data_on_disk = eseries.data
timestamps_on_disk = eseries.timestamps
n_dim = len(data_on_disk.shape)
n_samples = len(timestamps_on_disk)
        # find the
time_axis = 0 if data_on_disk.shape[0] == n_samples else 1
electrode_axis = 1 - time_axis
n_electrodes = data_on_disk.shape[electrode_axis]
input_dim_restrictions = [None] * n_dim

        # to get the input dimension restrictions we need to look at the electrode table for the eseries and get
        # the indices from that
input_dim_restrictions[electrode_axis] = np.s_[get_electrode_indices(eseries, electrode_ids)]

indices = []
output_shape_list = [0] * n_dim
output_shape_list[electrode_axis] = len(electrode_ids)
output_offsets = [0]

timestamp_size = timestamps_on_disk[0].itemsize
timestamp_dtype = timestamps_on_disk[0].dtype
data_size = data_on_disk[0][0].itemsize
data_dtype = data_on_disk[0][0].dtype
filter_delay = self.calc_filter_delay(filter_coeff)
        
for a_start, a_stop in valid_times:
    if a_start < timestamps_on_disk[0]:
        raise Warning('Interval start time %f is smaller than first timestamp %f, using first timestamp instead' % (
            a_start, timestamps_on_disk[0]))
        a_start = timestamps_on_disk[0]
    if a_stop > timestamps_on_disk[-1]:
        raise Warning('Interval stop time %f is larger than last timestamp %f, using last timestamp instead' % (
            a_stop, timestamps_on_disk[-1]))
        a_stop = timestamps_on_disk[-1]
    frm, to = np.searchsorted(timestamps_on_disk, (a_start, a_stop))
    if to > n_samples:
        to = n_samples
    indices.append((frm, to))
    shape, dtype = gsp.filter_data_fir(data_on_disk,
                                            filter_coeff,
                                            axis=time_axis,
                                            input_index_bounds=[frm, to-1],
                                            output_index_bounds=[
                                                filter_delay, filter_delay + to - frm],
                                            describe_dims=True,
                                            ds=decimation,
                                            input_dim_restrictions=input_dim_restrictions)
    output_offsets.append(output_offsets[-1] + shape[time_axis])
    output_shape_list[time_axis] += shape[time_axis]

        # open the nwb file to create the dynamic table region and electrode series, then write and close the file
with pynwb.NWBHDF5IO(path=analysis_file_abs_path, mode="a", load_namespaces=True) as io:
    nwbf = io.read()
            # get the indices of the electrodes in the electrode table
    elect_ind = get_electrode_indices(nwbf, electrode_ids)

    electrode_table_region = nwbf.create_electrode_table_region(
        elect_ind, 'filtered electrode table')
    eseries_name = 'filtered data'
    es = pynwb.ecephys.ElectricalSeries(name=eseries_name,
                                                data=np.empty(
                                                    tuple(output_shape_list), dtype=data_dtype),
                                                electrodes=electrode_table_region,
                                                timestamps=np.empty(output_shape_list[time_axis]))
            # Add the electrical series to the scratch area
    nwbf.add_scratch(es)
    io.write(nwbf)

            # reload the NWB file to get the h5py objects for the data and the timestamps
    with pynwb.NWBHDF5IO(path=analysis_file_abs_path, mode="a", load_namespaces=True) as io:
        nwbf = io.read()
        es = nwbf.objects[es.object_id]
        filtered_data = es.data
        new_timestamps = es.timestamps
        indices = np.array(indices, ndmin=2)
                # Filter and write the output dataset
        ts_offset = 0


        print('Filtering data')
        for ii, (start, stop) in enumerate(indices):
                    # calculate the size of the timestamps and the data and determine whether they
                    # can be loaded into < 90% of available RAM
            mem = psutil.virtual_memory()
            interval_samples = stop-start
            if interval_samples * (timestamp_size +  n_electrodes*data_size) < 0.9 * mem.available:
                print(f'Interval {ii}: loading data into memory')
                timestamps = np.asarray(timestamps_on_disk[start:stop], dtype=timestamp_dtype)
                if time_axis == 0:
                    data = np.asarray(data_on_disk[start:stop, :], dtype=data_dtype)
                else:
                    data = np.asarray(data_on_disk[:, start:stop], dtype=data_dtype)
                extracted_ts = timestamps[0::decimation]
                new_timestamps[ts_offset:ts_offset +
                        len(extracted_ts)] = extracted_ts
                ts_offset += len(extracted_ts)
                        # filter the data
                gsp.filter_data_fir(data,
                                            filter_coeff,
                                            axis=time_axis,
                                            input_index_bounds=[0, interval_samples-1],
                                            output_index_bounds=[
                                                filter_delay, filter_delay + stop - start],
                                            ds=decimation,
                                            input_dim_restrictions=input_dim_restrictions,
                                            outarray=filtered_data,
                                            output_offset=output_offsets[ii])
            else:
                print(f'Interval {ii}: leaving data on disk')
                data = data_on_disk
                timestamps = timestamps_on_disk
                extracted_ts = timestamps[start:stop:decimation]
                new_timestamps[ts_offset:ts_offset +
                                    len(extracted_ts)] = extracted_ts
                ts_offset += len(extracted_ts)
                        # filter the data
                gsp.filter_data_fir(data,
                                    filter_coeff,
                                    axis=time_axis,
                                    input_index_bounds=[start, stop],
                                    output_index_bounds=[
                                        filter_delay, filter_delay + stop - start],
                                    ds=decimation,
                                    input_dim_restrictions=input_dim_restrictions,
                                    outarray=filtered_data,
                                    output_offset=output_offsets[ii])

        start_end = [new_timestamps[0], new_timestamps[-1]]

        io.write(nwbf)

# return es.object_id, start_end

IndentationError: unexpected indent (968129783.py, line 42)